<a href="https://colab.research.google.com/github/thasliya0404/internship/blob/main/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/mergeddata (1).csv')

In [ ]:
import pandas as pd
import re

def clean_text(text):
    if pd.isna(text):
        return None
    # Convert to string to ensure string methods can be applied
    text = str(text)
    text = text.strip().lower()

    if text == "":
        return None  # remove empty feedback

    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text

data["CLEAN_REMARKS"] = data["REMARKS"].apply(clean_text)

In [ ]:
data.isna().sum()

,0
COURSE_ID,0
FEEDBACK_ID,0
KO,0
DATE,0
QUESTION_ID,0
VALUE,0
STUDENT_ID,0
TIMESTAMP,0
TYPE,0
REMARKS,0


In [ ]:
data = data.dropna(subset=['CLEAN_REMARKS'])

In [ ]:
!pip install gensim nltk
import nltk
nltk.download('stopwords')
 # Download punkt_tab as specifically requested by the error

from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def tokenize(text):
    if text is None:
        return []
    return [w for w in simple_preprocess(text) if w not in stop_words]

data["tokens"] = data["CLEAN_REMARKS"].apply(tokenize)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 74.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
MIN_TOKENS = 3
data["token_count"] = data["tokens"].apply(len)
df_lda = data[data["token_count"] >= MIN_TOKENS].reset_index(drop=True)

In [ ]:
df_lda = df_lda[df_lda['QUESTION_ID']==11]

In [ ]:
MIN_TOKENS = 3
data["token_count"] = data["tokens"].apply(len)
df_rest = data[data["token_count"] < MIN_TOKENS].reset_index(drop=True)

In [ ]:
df_rest = df_rest[df_rest['QUESTION_ID']==11]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install bertopic
from bertopic import BERTopic
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    min_samples=1,   # LOWER = fewer -1
    prediction_data=True
)

umap_model = UMAP(
    n_neighbors=50,    # ← increase from 30
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

topic_model = BERTopic(
    language="english",
    hdbscan_model = hdbscan_model,
    umap_model = umap_model
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.9 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
import re

def split_compound_sentences(text):
    """
    Split text into sentences on:
    - 'but', 'however', ';'
    - single full stop '.'
    Do NOT split if a number is on both sides of the dot (e.g., 12.30 or 3.14)
    """
    # Regex explanation:
    # \bbut\b|\bhowever\b|;        -> split on 'but', 'however', ';'
    # (?<!\d)\.(?!\d)             -> match a dot not preceded AND not followed by a digit
    pattern = (
    r'\bbut\b'
    r'|\bhowever\b'
    r'|\botherwise\b'
    r'|\bother than that\b'
    r'|;'
    r'|(?<!\d)(?<!\.)\.(?!\d)(?!\.)'
    )

    # Split using the pattern
    parts = re.split(pattern, text, flags=re.IGNORECASE)
    # Strip whitespace and remove empty strings
    return [p.strip() for p in parts if p.strip()]

In [ ]:
from nltk.tokenize import sent_tokenize
import pandas as pd
nltk.download('punkt')
nltk.download('punkt_tab')

# Ensure the 'REMARKS' column in data2 is of string type
df_lda['REMARKS'] = df_lda['REMARKS'].astype(str)

# Function to split feedback into sentences
df_lda['sentences'] = df_lda['REMARKS'].apply(split_compound_sentences)

# Flatten sentences into a list for BERTopic, using sentences from data2 corresponding to df_lda's index
# This ensures consistency with the df_lda filters applied previously.
all_sentences = [sent for sublist in df_lda.loc[df_lda.index, 'sentences'] for sent in sublist]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
topics, probs = topic_model.fit_transform(all_sentences)
# See topic info
topic_model.get_topic_info()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,515,-1_to_the_and_was,"[to, the, and, was, more, as, for, it, this, s...",[As many developers and non Devops people are ...
1,0,114,0_ai_tools_about_using,"[ai, tools, about, using, of, in, include, how...","[Include more tools related to AI, More practi..."
2,1,107,1_internet_connection_network_wifi,"[internet, connection, network, wifi, issue, n...","[Poor internet connection, Poor internet conne..."
3,2,81,2_sessions_practical_session_more,"[sessions, practical, session, more, need, lik...","[Need practical sessions, Need more practical ..."
4,3,81,3_break_breaks_snacks_lunch,"[break, breaks, snacks, lunch, give, time, tea...","[More time for lunch break, give more break in..."
5,4,76,4_teaching_way_mentor_he,"[teaching, way, mentor, he, keep, this, sir, s...","[Keep it this way of teaching, Good way of tea..."
6,5,74,5_class_good_nice_excellent,"[class, good, nice, excellent, very, wonderful...","[Good class, Good class, good class]"
7,6,63,6_activities_group_activity_more,"[activities, group, activity, more, interactiv...","[Extra group activities, More group activities..."
8,7,60,7_speed_bit_slow_pace,"[speed, bit, slow, pace, little, fast, slightl...","[The pace is a bit fast, Pace is bit slow, Cou..."
9,8,58,8_topics_topic_cover_more,"[topics, topic, cover, more, explain, to, unde...","[Need to cover more topics, Need more time to ..."


In [ ]:
df_lda = df_lda.explode("sentences")
df_lda["topic"] = topics

In [ ]:
df_lda

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS,CLEAN_REMARKS,tokens,token_count,sentences,topic
3,14,52,90.0,2024-09-28,11,0,129,2024-09-28 12:41:55.493,2,more Live case study is helpful,2,2,RETAIL,RETAIL,more live case study is helpful,"[live, case, study, helpful]",4,more Live case study is helpful,31
7,16,54,87.0,2024-09-28,11,0,144,2024-09-28 15:47:30.937,2,The class was good and nothing much thank you,6,4,ACADEMIC,ACADEMIC (SOUTH),the class was good and nothing much thank you,"[class, good, nothing, much, thank]",5,The class was good and nothing much thank you,23
10,16,54,87.0,2024-09-28,11,0,187,2024-09-28 15:48:14.533,2,Classes are long due to it it became boring,6,4,ACADEMIC,ACADEMIC (SOUTH),classes are long due to it it became boring,"[classes, long, due, became, boring]",5,Classes are long due to it it became boring,11
17,16,54,87.0,2024-09-28,11,0,149,2024-09-28 15:50:11.183,2,This team is good and we believe that ith will...,6,4,ACADEMIC,ACADEMIC (SOUTH),this team is good and we believe that ith will...,"[team, good, believe, ith, improve, coming, ye...",8,This team is good and we believe that ith will...,27
19,16,54,87.0,2024-09-28,11,0,145,2024-09-28 15:50:27.107,2,The session should be a lot bit faster,6,4,ACADEMIC,ACADEMIC (SOUTH),the session should be a lot bit faster,"[session, lot, bit, faster]",4,The session should be a lot bit faster,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11550,1252,1309,87.0,2025-08-26,11,0,25817,2025-08-26 16:09:47.613,2,Devices provided for practice was not supporti...,1,1,GOVERNMENT,GOVERNMENT,devices provided for practice was not supporti...,"[devices, provided, practice, supporting, late...",7,Devices provided for practice was not supporti...,48
11553,1252,1309,87.0,2025-08-26,11,0,25823,2025-08-26 16:11:54.120,2,ADD MORE TOOLS AND MORE SYLLABUS,1,1,GOVERNMENT,GOVERNMENT,add more tools and more syllabus,"[add, tools, syllabus]",3,ADD MORE TOOLS AND MORE SYLLABUS,48
11556,1252,1309,87.0,2025-08-26,11,0,25827,2025-08-26 16:12:28.747,2,Add more days and cover more tool and sylabus,1,1,GOVERNMENT,GOVERNMENT,add more days and cover more tool and sylabus,"[add, days, cover, tool, sylabus]",5,Add more days and cover more tool and sylabus,9
11559,1252,1309,87.0,2025-08-26,11,0,25807,2025-08-26 16:09:55.600,2,AI tools for cyber security,1,1,GOVERNMENT,GOVERNMENT,ai tools for cyber security,"[ai, tools, cyber, security]",4,AI tools for cyber security,0


In [ ]:
df_rest.shape

(17431, 17)

In [ ]:
df_rest['REMARKS'] = df_rest['REMARKS'].astype(str)

# Function to split feedback into sentences
df_rest['sentences'] = df_rest['REMARKS'].apply(split_compound_sentences)

# Flatten sentences into a list for BERTopic, using sentences from data2 corresponding to df_lda's index
# This ensures consistency with the df_lda filters applied previously.

In [ ]:
JUNK = {
    np.number, ".", "..", "", np.nan, '-1', 'nil', 'na', 'null', 'no', 'nop', 'noo', 'good', '-'
}

def is_valid_sentence(s):
    s = s.strip().lower()
    if s in JUNK:
        return False
    return True

In [ ]:
df_rest = df_rest.explode('sentences')

In [ ]:
df_rest.shape

(17447, 18)

In [ ]:
df_rest["sentences"] = df_rest["sentences"].fillna("")
df_rest["valid_for_topic"] = df_rest["sentences"].apply(is_valid_sentence)
df_valid = df_rest[df_rest["valid_for_topic"]]

In [ ]:
df_valid

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS,CLEAN_REMARKS,tokens,token_count,sentences,valid_for_topic
3,12,50,92.0,2024-09-24,11,0,113,2024-09-24 13:05:35.310,2,Notjing spl,2,2,RETAIL,RETAIL,notjing spl,"[notjing, spl]",2,Notjing spl,True
16,12,50,92.0,2024-09-24,11,0,0,2024-09-24 13:13:19.123,2,Excellent,2,2,RETAIL,RETAIL,excellent,[excellent],1,Excellent,True
28,16,54,87.0,2024-10-05,11,0,169,2024-10-05 15:41:50.030,2,No other suggestions,6,4,ACADEMIC,ACADEMIC (SOUTH),no other suggestions,[suggestions],1,No other suggestions,True
56,16,54,87.0,2024-09-28,11,0,174,2024-09-28 15:47:00.827,2,Nothing,6,4,ACADEMIC,ACADEMIC (SOUTH),nothing,[nothing],1,Nothing,True
76,16,54,87.0,2024-09-28,11,0,175,2024-09-28 15:49:30.407,2,Nothing,6,4,ACADEMIC,ACADEMIC (SOUTH),nothing,[nothing],1,Nothing,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46832,1252,1309,87.0,2025-08-26,11,0,25831,2025-08-26 16:09:33.117,2,Nothing,1,1,GOVERNMENT,GOVERNMENT,nothing,[nothing],1,Nothing,True
46835,1252,1309,87.0,2025-08-26,11,0,25820,2025-08-26 16:12:12.727,2,Overall good.,1,1,GOVERNMENT,GOVERNMENT,overall good,"[overall, good]",2,Overall good,True
46840,1252,1309,87.0,2025-08-26,11,0,25819,2025-08-26 16:13:43.833,2,Very useful in future,1,1,GOVERNMENT,GOVERNMENT,very useful in future,"[useful, future]",2,Very useful in future,True
46844,1237,1286,88.0,2025-08-27,11,0,15205,2025-08-27 17:34:51.087,2,None,2,2,RETAIL,RETAIL,none,[none],1,None,True


In [ ]:
df_valid.shape

(6676, 19)

In [ ]:
# Get the list of valid sentences directly from df_valid
valid_sentences = df_valid['sentences'].tolist()

# Transform these sentences to get the topics
topics_for_df_valid, probs_for_df_valid = topic_model.transform(valid_sentences)

# Assign the correctly generated topics to df_valid
df_valid["topic"] = topics_for_df_valid

In [ ]:
df_valid.columns

Index(['COURSE_ID', 'FEEDBACK_ID', 'KO', 'DATE', 'QUESTION_ID', 'VALUE',
       'STUDENT_ID', 'TIMESTAMP', 'TYPE', 'REMARKS', 'SUBOU_ID', 'OU_ID', 'OU',
       'OUS', 'CLEAN_REMARKS', 'tokens', 'token_count', 'sentences',
       'valid_for_topic', 'topic'],
      dtype='object')

In [ ]:
env_topics = []   # internet_network_wifi

df_select = df_valid[df_valid["topic"].isin(env_topics)]
df_select['sentences'].count()

np.int64(0)

In [ ]:
df_valid.head(50)

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS,CLEAN_REMARKS,tokens,token_count,sentences,valid_for_topic,topic
3,12,50,92.0,2024-09-24,11,0,113,2024-09-24 13:05:35.310,2,Notjing spl,2,2,RETAIL,RETAIL,notjing spl,"[notjing, spl]",2,Notjing spl,True,-1
16,12,50,92.0,2024-09-24,11,0,0,2024-09-24 13:13:19.123,2,Excellent,2,2,RETAIL,RETAIL,excellent,[excellent],1,Excellent,True,24
28,16,54,87.0,2024-10-05,11,0,169,2024-10-05 15:41:50.030,2,No other suggestions,6,4,ACADEMIC,ACADEMIC (SOUTH),no other suggestions,[suggestions],1,No other suggestions,True,44
56,16,54,87.0,2024-09-28,11,0,174,2024-09-28 15:47:00.827,2,Nothing,6,4,ACADEMIC,ACADEMIC (SOUTH),nothing,[nothing],1,Nothing,True,38
76,16,54,87.0,2024-09-28,11,0,175,2024-09-28 15:49:30.407,2,Nothing,6,4,ACADEMIC,ACADEMIC (SOUTH),nothing,[nothing],1,Nothing,True,38
78,16,54,87.0,2024-09-28,11,0,146,2024-09-28 15:49:40.660,2,Speaker,6,4,ACADEMIC,ACADEMIC (SOUTH),speaker,[speaker],1,Speaker,True,29
104,14,52,97.0,2024-10-16,11,0,125,2024-10-16 13:34:13.087,2,Understandable,2,2,RETAIL,RETAIL,understandable,[understandable],1,Understandable,True,-1
121,17,55,87.0,2024-10-17,11,0,201,2024-10-17 21:54:55.873,2,Overall good,2,2,RETAIL,RETAIL,overall good,"[overall, good]",2,Overall good,True,24
127,16,54,87.0,2024-10-05,11,0,152,2024-10-05 15:42:19.567,2,Can zoom a little,6,4,ACADEMIC,ACADEMIC (SOUTH),can zoom a little,"[zoom, little]",2,Can zoom a little,True,17
148,16,54,87.0,2024-10-19,11,0,150,2024-10-19 15:48:16.830,2,Nothing,6,4,ACADEMIC,ACADEMIC (SOUTH),nothing,[nothing],1,Nothing,True,38


In [ ]:
df_tranformed = pd.merge(
    left=df_rest,
    right=df_valid,
    how='left',
    on=['TIMESTAMP', 'sentences', 'COURSE_ID', 'FEEDBACK_ID', 'KO', 'DATE', 'QUESTION_ID', 'VALUE', 'STUDENT_ID', 'TYPE', 'REMARKS', 'SUBOU_ID', 'OU_ID', 'OU', 'OUS', 'CLEAN_REMARKS', 'token_count',
        'valid_for_topic']
)

In [ ]:
df_tranformed.shape

(17447, 21)

In [ ]:
df_11 = pd.concat([df_tranformed, df_lda], ignore_index=True, sort=False)

In [ ]:
df_11.shape

(19620, 22)

In [ ]:
df_11.columns

Index(['COURSE_ID', 'FEEDBACK_ID', 'KO', 'DATE', 'QUESTION_ID', 'VALUE',
       'STUDENT_ID', 'TIMESTAMP', 'TYPE', 'REMARKS', 'SUBOU_ID', 'OU_ID', 'OU',
       'OUS', 'CLEAN_REMARKS', 'tokens_x', 'token_count', 'sentences',
       'valid_for_topic', 'tokens_y', 'topic', 'tokens'],
      dtype='object')

In [ ]:
df_not11 = data[data['QUESTION_ID']!=11]

In [ ]:
df_all = pd.concat([df_11, df_not11], ignore_index=True, sort=False)

In [ ]:
df_all.shape

(58446, 22)

In [ ]:
df_not11.shape

(38826, 17)

In [ ]:
df_lda.shape

(2173, 18)

In [ ]:
topic_model.save("bertopic_model1")

2025-12-30 20:22:30,759 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
data[data['QUESTION_ID']==11]

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS,CLEAN_REMARKS,tokens,token_count
1,12,50,92.0,2024-09-24,11,0,93,2024-09-24 10:45:38.227,2,Nil,2,2,RETAIL,RETAIL,nil,[nil],1
4,12,50,92.0,2024-09-24,11,0,113,2024-09-24 13:05:35.310,2,Notjing spl,2,2,RETAIL,RETAIL,notjing spl,"[notjing, spl]",2
6,12,50,92.0,2024-09-24,11,0,0,2024-09-24 13:05:44.417,2,Nop,2,2,RETAIL,RETAIL,nop,[nop],1
8,12,50,92.0,2024-09-24,11,0,96,2024-09-24 13:05:59.907,2,-1,2,2,RETAIL,RETAIL,,[],0
10,12,50,92.0,2024-09-24,11,0,65,2024-09-24 13:06:14.830,2,-1,2,2,RETAIL,RETAIL,,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58602,1237,1286,88.0,2025-08-27,11,0,15206,2025-08-27 17:36:09.800,2,Nothing,2,2,RETAIL,RETAIL,nothing,[nothing],1
58605,1237,1286,88.0,2025-08-27,11,0,15215,2025-08-27 17:36:11.820,2,Nil,2,2,RETAIL,RETAIL,nil,[nil],1
58608,1204,1250,96.0,2025-08-28,11,0,14302,2025-08-28 13:15:05.043,2,.,2,2,RETAIL,RETAIL,,[],0
58611,1254,1310,87.0,2025-08-29,11,0,25833,2025-08-29 16:48:53.683,2,No,1,1,GOVERNMENT,GOVERNMENT,no,[],0


In [ ]:
df_11 = df_11.drop(['CLEAN_REMARKS', 'tokens_x', 'token_count', 'valid_for_topic', 'tokens_y', 'tokens'], axis=1)

In [ ]:
df_11.columns

Index(['COURSE_ID', 'FEEDBACK_ID', 'KO', 'DATE', 'QUESTION_ID', 'VALUE',
       'STUDENT_ID', 'TIMESTAMP', 'TYPE', 'REMARKS', 'SUBOU_ID', 'OU_ID', 'OU',
       'OUS', 'sentences', 'topic'],
      dtype='object')

In [ ]:
df_11.to_csv('data_q11.csv', index=False)

In [ ]:
env_topics = [26]   # internet_network_wifi

df_select = df_lda[df_lda["topic"].isin(env_topics)]
list(df_select['sentences'])

['AC kurave aanu',
 'Would like to get an easy access to all facilities like Paatshala platform, linkedin learning etc',
 'Didnt get acess to paatshala',
 'please take the class in malayalam language for more convenienence',
 '....Adichu keeri vaa mwone....',
 'ac korachal kollam',
 'Kurchu kudi nerthe videnu',
 'Kurch kudi nerthe vidanam',
 'Nammk kurach slow aayi paranjtharanm,karanm njnghal 1st yr aan',
 'Njngal 1st yrsine korach kudi onn care akkane sir',
 'It will be more good to explain in malayalam',
 'If the classes are in malayalam',
 'Faculties are too good, there are some misunderstndings regarding paatshala platform',
 'code explain in malayalam like today section',
 'Please fix the Paatshala errors',
 'Korachu koda slow ell parnjal nanayerikum',
 'korach friendly avam',
 'session edukkunna speed kurach kurakkam',
 'examm easy  akkaneeeeeee',
 'Sir Aswin Game Kalikuvaaa..so Avane Oru PC Kodukkanmm GAminginu',
 'I couldnt see any of the recorded class in Paatshala',
 'Certif

* 0 - Sessions related to AI
* 1 - Internet, Wi-Fi, and Network Connectivity Issues
* 2, 22 - Need for More Practical, Hands-on, and Extended Sessions
* 3 - Insufficient Break Time and Need for More Frequent Intervals
* 4, 40 - Teaching methodology & instructor effectiveness
* 5, 14, 16, 20, 21, 23, 24, 32, 33, 37, 39, 41, 46, 53, 50  - Overall Positive Feedback
* 6 - Activities & engagement
* 7, 10, 45 - Session Pace and Speed of Delivery
* 8 - Topic Coverage, Depth, and Time Allocation
* 9, 11, 28, 43, 52 - Session Duration, Intervals, and Time Allocation
* 10 - Need for Advanced, Updated, and Job-Relevant Tools
* 11 - Overall Satisfaction with No Suggestions for Improvement
* 12, 34 - Need for Conceptual Clarity, Basic Explanations, and Learning Resources
* 13 - Interest in Continued and Advanced Skill-Based Courses
* 14 - Classroom Infrastructure and Temperature Comfort
* 15 - Expectations for Practical Data Analytics and Office-Oriented Tools
* 17 - Classroom Infrastructure and Projector Visibility Issues
* 18 - Classroom Thermal Comfort and Air Conditioning Issues
* 19, 31, 48, 49 - Need for Advanced, Practical, and Extended Training Programs
* 22, 55 - Interaction and Engagement in Learning
* 25 - Trainer Effectiveness and Teaching Approach
* 26 - Malayalam
* 27 - Improvement, and Minor Operational Suggestions
* 29 - Audio Quality and Classroom Environment
* 30, 54 - Coding Related Queries
* 35 - Content Delivery and Section Quality
* 36 - Information and Effectiveness of the session
* 38, 44 - Nothing, No Suggestion
* 42, 47 - Study Material
* 51 - Presentation and slides Quality
* 56 - Sitting capacity

In [ ]:
feedback_topics = {
    "Sessions related to AI": [0],
    "Internet, Wi-Fi, and Network Connectivity Issues": [1],
    "Need for More Practical, Advanced, Hands-on, and Extended Sessions": [2, 22, 10, 19, 31, 48, 49],
    "Insufficient Break Time and Need for More Frequent Intervals": [3],
    "Teaching methodology & instructor effectiveness": [4, 40, 25],
    "Overall Positive Feedback": [5, 14, 16, 20, 21, 23, 24, 32, 33, 37, 39, 41, 46, 50, 53],
    "Session Pace and Speed of Delivery": [7, 10, 45],
    "Topic Coverage, Depth, and Time Allocation": [8],
    "Session Duration, Intervals, and Time Allocation": [9, 11, 28, 43, 52],
    "Overall Satisfaction with No Suggestions for Improvement": [11],
    "Need for Conceptual Clarity, Basic Explanations, and Learning Resources": [12, 34],
    "Interest in Continued and Advanced Skill-Based Courses": [13],
    "Classroom Infrastructure and Temperature Comfort": [14, 18],
    "Expectations for Practical Data Analytics and Office-Oriented Tools": [15],
    "Classroom Infrastructure and Projector Visibility Issues": [17],
    "Classroom Thermal Comfort and Air Conditioning Issues": [18],
    "Interaction, Activities and Engagement in Learning": [22, 55, 6],
    "Malayalam": [26],
    "Improvement and Minor Operational Suggestions": [27],
    "Audio Quality and Classroom Environment": [29],
    "Coding Related Queries": [30, 54],
    "Content Delivery and Section Quality": [35],
    "Information and Effectiveness of the session": [36],
    "Nothing, No Suggestion": [38, 44],
    "Study Material": [42, 47],
    "Presentation and slides Quality": [51],
    "Sitting capacity": [56]
}


In [ ]:
import pandas as pd
import numpy as np

# List of "empty feedback" words
empty_comments = ['nil', 'na', 'null', 'no', 'nop', 'noo', 'nil']

def map_topic_conditional(row):
    if pd.notna(row['topic']):  # if topic exists
        for group, ids in feedback_topics.items():
            if row['topic'] in ids:
                return group
        return "Other"  # topic not in dictionary
    else:  # topic is NaN
        # check if sentence contains any empty comments
        sentence_lower = str(row['sentences']).lower()
        if any(word in sentence_lower for word in empty_comments):
            return "Nothing, No Suggestion"
        else:
            return "Other"
# Apply function row-wise
df_11['topic_group'] = df_11.apply(map_topic_conditional, axis=1)


In [ ]:
df_11.to_csv('data_11.csv')

In [ ]:
df_11.to_csv('data_q11_topic.csv', index=False)

In [ ]:
df_11[df_11['topic_group']!="Other"]

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS,sentences,topic,topic_group
0,12,50,92.0,2024-09-24,11,0,93,2024-09-24 10:45:38.227,2,Nil,2,2,RETAIL,RETAIL,Nil,NaN,"Nothing, No Suggestion"
2,12,50,92.0,2024-09-24,11,0,0,2024-09-24 13:05:44.417,2,Nop,2,2,RETAIL,RETAIL,Nop,NaN,"Nothing, No Suggestion"
8,12,50,92.0,2024-09-24,11,0,0,2024-09-24 13:13:19.123,2,Excellent,2,2,RETAIL,RETAIL,Excellent,24.0,Overall Positive Feedback
14,16,54,87.0,2024-10-05,11,0,169,2024-10-05 15:41:50.030,2,No other suggestions,6,4,ACADEMIC,ACADEMIC (SOUTH),No other suggestions,44.0,"Nothing, No Suggestion"
27,16,54,87.0,2024-09-28,11,0,157,2024-09-28 15:46:59.607,2,Nil,6,4,ACADEMIC,ACADEMIC (SOUTH),Nil,NaN,"Nothing, No Suggestion"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19594,1251,1302,168.0,2025-08-20,11,0,15560,2025-08-20 15:44:43.017,2,Eventhough the group coordination was not too ...,5,4,ACADEMIC,ACADEMIC (CENTRAL),[Eventhough the group coordination was not too...,NaN,"Nothing, No Suggestion"
19602,1251,1300,108.0,2025-08-20,11,0,15527,2025-08-20 15:48:28.400,2,Nothing...it was really good,5,4,ACADEMIC,ACADEMIC (CENTRAL),[Nothing...it was really good],NaN,"Nothing, No Suggestion"
19610,1252,1309,87.0,2025-08-26,11,0,25803,2025-08-26 16:08:08.807,2,Check every systems in the training nalla befo...,1,1,GOVERNMENT,GOVERNMENT,[Check every systems in the training nalla bef...,NaN,"Nothing, No Suggestion"
19612,1252,1309,87.0,2025-08-26,11,0,25821,2025-08-26 16:08:32.593,2,class duration is not sufficient,1,1,GOVERNMENT,GOVERNMENT,[class duration is not sufficient],NaN,"Nothing, No Suggestion"


In [ ]:
df_lda.shape

(2540, 19)